In [24]:
import sqlite3
import pandas as pd
import numpy as np
import random

In [41]:
# Establish connection
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

In [42]:
# Creating Books Table
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Books (
    BookID INTEGER PRIMARY KEY AUTOINCREMENT,
    Title TEXT NOT NULL,
    Author TEXT NOT NULL,
    Genre TEXT CHECK(Genre IN ('Fiction', 'Non-Fiction', 'Science', 'History', 'Biography')) NOT NULL,
    PublishedYear INTEGER NOT NULL,
    ISBN TEXT UNIQUE NOT NULL
);
""")

cursor.executescript("""
-- Creating Members Table
CREATE TABLE IF NOT EXISTS Members (
    MemberID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Email TEXT UNIQUE NULL,
    PhoneNumber TEXT NULL,
    RegistrationDate DATE NULL
);
""")

cursor.executescript("""
-- Creating BorrowedBooks Table
CREATE TABLE IF NOT EXISTS BorrowedBooks (
    BorrowID INTEGER PRIMARY KEY AUTOINCREMENT,
    MemberID INTEGER NOT NULL,
    BookID INTEGER NOT NULL,
    BorrowDate DATETIME NOT NULL,
    DueDate DATETIME NOT NULL,
    ReturnDate DATETIME,
    FOREIGN KEY (MemberID) REFERENCES Members(MemberID),
    FOREIGN KEY (BookID) REFERENCES Books(BookID)
);
""")

cursor.executescript("""
-- Creating Staff Table
CREATE TABLE IF NOT EXISTS Staff (
    StaffID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Role TEXT CHECK(Role IN ('Librarian', 'Assistant', 'Manager')) NOT NULL,
    Salary REAL NULL,
    HireDate DATE NULL
);
""")

In [43]:
# Insert Sample Data
cursor.executescript("""
INSERT INTO Books (Title, Author, Genre, PublishedYear, ISBN) VALUES
('The Great Gatsby', 'F. Scott Fitzgerald', 'Fiction', 1925, '9780743273565'),
('A Brief History of Time', 'Stephen Hawking', 'Science', 1988, '9780553380163');
""")

cursor.executescript("""
INSERT INTO Members (Name, Email, PhoneNumber, RegistrationDate) VALUES
('Alice Johnson', 'alice@example.com', '123-456-7890', '2023-01-10'),
('Bob Smith', 'bob@example.com', '321-654-0987', '2022-12-05');
""")

cursor.executescript("""
INSERT INTO BorrowedBooks (MemberID, BookID, BorrowDate, DueDate, ReturnDate) VALUES
(1, 1, '2025-03-10 10:00:00', '2025-03-20 10:00:00', NULL);
""")

cursor.executescript("""
INSERT INTO Staff (Name, Role, Salary, HireDate) VALUES
('Emily Davis', 'Librarian', 35000, '2021-06-15'),
('Michael Brown', 'Assistant', 28000, '2022-08-20');
""")

In [44]:
# List of Names
names = ["Alice Johnson", "Bob Smith", "Charlie Davis", "Diana Roberts", "Ethan Clark", "Fiona Lee", "George Wilson", "Hannah Brown", "Ian White", "Jessica Adams"]

In [45]:
# Function to generate random data
def generate_data(num_records, table):
    np.random.seed(42)
    if table == "Books":
        books = [
            (f"Book {i if random.random() > 0.1 else 1}", random.choice(names), random.choice(["Fiction", "Non-Fiction", "Science", "History", "Biography"]),
             np.random.randint(1900, 2025), f"978-{random.randint(100000000, 999999999)}")
            for i in range(num_records)
        ]
        cursor.executemany("INSERT INTO Books (Title, Author, Genre, PublishedYear, ISBN) VALUES (?, ?, ?, ?, ?);", books)
    elif table == "Members":
        members = [
            (random.choice(names), f"member{i}@example.com" if random.random() > 0.1 else None, f"{random.randint(1000000000, 9999999999)}" if random.random() > 0.2 else None, f"202{random.randint(0, 4)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}" if random.random() > 0.1 else None)
            for i in range(num_records)
        ]
        cursor.executemany("INSERT INTO Members (Name, Email, PhoneNumber, RegistrationDate) VALUES (?, ?, ?, ?);", members)
    elif table == "BorrowedBooks":
        borrowed_books = [
            (random.randint(1, num_records), random.randint(1, num_records), f"202{random.randint(0, 4)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}",
             f"202{random.randint(0, 4)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}", None if random.random() > 0.2 else f"202{random.randint(0, 4)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}")
            for _ in range(num_records)
        ]
        cursor.executemany("INSERT INTO BorrowedBooks (MemberID, BookID, BorrowDate, DueDate, ReturnDate) VALUES (?, ?, ?, ?, ?);", borrowed_books)
    elif table == "Staff":
        staff = [
            (random.choice(names), random.choice(["Librarian", "Assistant", "Manager"]), np.random.randint(25000, 60000) if random.random() > 0.1 else None, f"201{random.randint(0, 9)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}" if random.random() > 0.1 else None)
            for i in range(num_records)
        ]
        cursor.executemany("INSERT INTO Staff (Name, Role, Salary, HireDate) VALUES (?, ?, ?, ?);", staff)


In [46]:
# Generate random data
generate_data(100, "Books")
generate_data(50, "Members")
generate_data(200, "BorrowedBooks")
generate_data(20, "Staff")

In [47]:
# Function to Run Queries and Display Results
def run_query(query):
    df = pd.read_sql_query(query, conn)
    return df

In [8]:
# Retrieve first 10 books
run_query("SELECT * FROM Books LIMIT 10;")

,BookID,Title,Author,Genre,PublishedYear,ISBN
0,1,The Great Gatsby,F. Scott Fitzgerald,Fiction,1925,9780743273565
1,2,A Brief History of Time,Stephen Hawking,Science,1988,9780553380163
2,3,Book 0,Author 0,Biography,2002,978-898224743
3,4,Book 1,Author 1,Biography,1951,978-516181491
4,5,Book 2,Author 2,Biography,1992,978-528789275
5,6,Book 3,Author 3,Biography,1914,978-444339979
6,7,Book 4,Author 4,Science,2006,978-733915333
7,8,Book 5,Author 5,Fiction,1971,978-790245186
8,9,Book 6,Author 6,Fiction,1960,978-168378561
9,10,Book 7,Author 7,Fiction,1920,978-502694089


In [48]:
# Get members who registered after 2023
run_query("SELECT Name, Email FROM Members WHERE RegistrationDate > '2023-01-01';")

,Name,Email
0,Alice Johnson,alice@example.com
1,George Wilson,member1@example.com
2,Jessica Adams,member3@example.com
3,Charlie Davis,member4@example.com
4,Diana Roberts,member5@example.com
5,Alice Johnson,member8@example.com
6,George Wilson,member9@example.com
7,Ian White,member11@example.com
8,Ian White,member14@example.com
9,Alice Johnson,member19@example.com


In [49]:
# Get book titles along with the members who borrowed them
run_query("""SELECT Books.Title, Members.Name FROM BorrowedBooks
INNER JOIN Books ON BorrowedBooks.BookID = Books.BookID
INNER JOIN Members ON BorrowedBooks.MemberID = Members.MemberID LIMIT 10;
""")

,Title,Name
0,The Great Gatsby,Alice Johnson
1,Book 96,Alice Johnson
2,Book 61,Charlie Davis
3,Book 93,Alice Johnson
4,Book 39,Diana Roberts
5,Book 60,Bob Smith
6,Book 17,Diana Roberts
7,Book 6,Alice Johnson
8,Book 79,Charlie Davis
9,Book 15,Charlie Davis


In [50]:
# Count staff by role
run_query("SELECT Role, COUNT(*) as Total FROM Staff GROUP BY Role;")

,Role,Total
0,Assistant,6
1,Librarian,9
2,Manager,7


In [51]:
conn.commit()
conn.close()